In [161]:
import pandas as pd
from datetime import datetime, timedelta
import os
import numpy as np
import duckdb as dd
from pathlib import Path

In [162]:
path = r"C:\Users\Akhil\Downloads\Total_D"

## Find the possiblity of common definitions

In [163]:
xlsx = [f for f in os.listdir(path) if f.endswith('.xlsx') and f.startswith('Export')]
if len(xlsx) != 3:
    raise ValueError(" The no of files are not exactly 3")
else:
    file_route_list = [os.path.join(path, f) for f in xlsx]
    stringee = pd.concat((pd.read_excel(f, engine = "openpyxl") for f in file_route_list), ignore_index=True)
stringee = stringee[stringee['End of Call code'] != 'CAN_NOT_MAKE_CALL']
stringee['Start time'] = pd.to_datetime(stringee['Start time'], format='%Y-%m-%d %H:%M:%S.%f',errors='coerce') + pd.Timedelta(hours=1, minutes=30)
stringee['Start time'] = stringee['Start time'].dt.strftime('%Y/%m/%d %H:%M:%S')
stringee["Queue Duration (timedelta)"]  = pd.to_timedelta(stringee["Queue duration"])
stringee["Answer Duration (timedelta)"] = pd.to_timedelta(stringee["Answer duration"])
stringee["Total Duration (timedelta)"] = (stringee["Queue Duration (timedelta)"] + stringee["Answer Duration (timedelta)"])
stringee["Total Duration"] = stringee["Total Duration (timedelta)"].dt.components.apply(lambda r: f"{int(r['hours']):02}:{int(r['minutes']):02}:{int(r['seconds']):02}", axis=1)
new_string = stringee[['Start time','Account','Call status','Answer duration','Hold duration','Total Duration','Customer number']]
new_string['Start time'] = pd.to_datetime(new_string['Start time'], errors='coerce', format='%Y/%m/%d %H:%M:%S')
new_string['Date'] = new_string['Start time'].dt.date
new_string['Call Start Time'] = new_string['Start time'].dt.strftime('%H:%M:%S')
new_string['Source'] = 'Stringee'

c:\Users\Akhil\Office\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


c:\Users\Akhil\Office\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Akhil\Office\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\Akhil\AppData\Local\Temp\ipykernel_23696\3518384511.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_string['Start time'] = pd.to_datetime(new_string['Start time'], errors='coerce', format='%Y/%m/%d %H:%M:%S')
C:\Users\Akhil\AppData\Local\Temp\ipykernel_23696\3518384511.py:16: SettingWithCopyWarnin

In [164]:
tata_1 = [f for f in os.listdir(path) if f.startswith("tata_dialer_call_report_")][0]
tata_2 = [f for f in os.listdir(path) if f.startswith("call_report_")][0]
T1 = pd.read_csv(os.path.join(path, tata_1))
T2 = pd.read_csv(os.path.join(path, tata_2))

In [165]:
T2 = T2.dropna(subset=["Call Flow"])
T2 = T2.rename(columns={"Client Number": "Customer Number", "Status": "Call Status"})
T2["Connected to Agent"]= T2["Call Flow"].str.extract(r'Agent:\s*([^-\(]+)')
T2["Call Start Date"] = T2["Call Flow"].str.extract(r'\((\d{2}-\d{2}-\d{4})\s')
T2["Call Start Time"] = T2["Call Flow"].str.extract(r'\d{4}\s(\d{2}:\d{2}:\d{2})')
T2 = T2.dropna(subset=["Call Start Date"])
T2["Customer Number"] = T2["Call Flow"].str.extract(r'PJSIP/(\+?\d+)@')
T2["Answer Duration (HH:MM:SS)"] = ( pd.to_timedelta(T2["Outbound Duration"].astype(float), unit="s") .astype(str) .str[-8:] .replace("NaT", "00:00:00") )
T2["Total Call Duration (HH:MM:SS)"] = ( pd.to_timedelta(T2["Call Duration"].astype(float), unit="s") .astype(str) .str[-8:] .replace("NaT", "00:00:00") )
T2["Hold Duration (HH:MM:SS)"] = ( pd.Series([pd.to_timedelta(0, unit="s")] * len(T2)) .dt.total_seconds() .astype(int) .apply(lambda x: f"{x//3600:02d}:{(x%3600)//60:02d}:{x%60:02d}") )
T2["Call Start Date"] = pd.to_datetime(T2["Call Start Date"], format='%d-%m-%Y') 
T2["Call Start Date"] = T2["Call Start Date"].dt.strftime('%Y-%m-%d')
cols = [ 'Call Start Date', 'Connected to Agent', 'Customer Number', 'Call Status', 'Answer Duration (HH:MM:SS)','Hold Duration (HH:MM:SS)',  'Total Call Duration (HH:MM:SS)', 'Call Start Time'] 
T2 = T2[cols] 
T1 = T1[cols]
tata = pd.concat([T1, T2], ignore_index=True)
new_tata = tata[['Call Start Date', 'Connected to Agent','Call Status','Answer Duration (HH:MM:SS)','Hold Duration (HH:MM:SS)','Total Call Duration (HH:MM:SS)','Call Start Time','Customer Number']]
new_tata['Source'] = 'Tata'

C:\Users\Akhil\AppData\Local\Temp\ipykernel_23696\1795672719.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_tata['Source'] = 'Tata'


In [166]:
know_r= [f for f in os.listdir(path) if f.startswith("call_logs_report_") and f.endswith(".csv")]
know = pd.read_csv(os.path.join(path, know_r[0]))
new_Know = know[['Date and Time', 'Agent Name','Call Status', 'Talk Time (hh:mm:ss)', 'Hold Time (hh:mm:ss)','Total Call Duration (hh:mm:ss)','Customer']]
new_Know['Date and Time'] = pd.to_datetime(new_Know['Date and Time'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
new_Know['Date'] = new_Know['Date and Time'].dt.date
new_Know['Call Start Time'] = new_Know['Date and Time'].dt.strftime('%H:%M:%S')
new_Know['Source'] = 'Knowlarity'

C:\Users\Akhil\AppData\Local\Temp\ipykernel_23696\1499990175.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_Know['Date and Time'] = pd.to_datetime(new_Know['Date and Time'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
C:\Users\Akhil\AppData\Local\Temp\ipykernel_23696\1499990175.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_Know['Date'] = new_Know['Date and Time'].dt.date
C:\Users\Akhil\AppData\Local\Temp\ipykernel_23696\1499990175.py:6: SettingWithCopyWarning: 
A value is trying to be se

## *try to normalize the column headers by making it lower*

In [167]:

column_mapping = {
    'Call Start Date': 'Date',
    'Connected to Agent': 'Dialer Name', 
    'Customer Number': 'Number',
    'Customer number' : 'Number',
    'Customer': 'Number',
    'Account': 'Dialer Name',
    'Agent Name': 'Dialer Name',
    'Call Status': 'Call Status',
    'Call status': 'Call Status',
    'Answer Duration (HH:MM:SS)': 'Talk Time',
    'Talk Time (hh:mm:ss)': 'Talk Time',
    'Answer duration': 'Talk Time',
    'Hold Duration (HH:MM:SS)': 'Hold Time',
    'Hold Time (hh:mm:ss)': 'Hold Time', 
    'Hold duration': 'Hold Time',
    'Total Call Duration (HH:MM:SS)': 'Total Call Duration',
    'Total Call Duration (hh:mm:ss)': 'Total Call Duration',
    'Total Duration': 'Total Call Duration',
    'Call Start Time': 'Call Start Time'
}

for df in [new_tata, new_Know, new_string]:
    df.rename(columns=column_mapping, inplace=True)

target_cols = ['Source','Date','Dialer Name','Number','Call Status',
               'Call Start Time','Total Call Duration','Talk Time','Hold Time']

selected_dfs = [df[target_cols] for df in [new_tata, new_Know, new_string]]
combined = pd.concat(selected_dfs, ignore_index=True)

C:\Users\Akhil\AppData\Local\Temp\ipykernel_23696\2780754403.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns=column_mapping, inplace=True)


In [169]:
# Refine the regex to only remove unwanted patterns
combined['Dialer Name'] = combined['Dialer Name'].str.replace(r"\s*\([^)]*\)|@.*|;.*", "", regex=True)

# Fill missing 'Dialer Name' values with their original values if they were numeric
combined['Dialer Name'] = combined['Dialer Name'].fillna(combined['Dialer Name'])

## Check here whether the fill na in the tata hold time could be removed

In [170]:
combined['Hold Time'] = combined['Hold Time'].fillna('00:00:00')

In [171]:
combined['Call Status'] = np.where(combined['Call Status'].astype(str).str.lower().eq('answered'), 'connected', 'not connected')
combined = combined[~combined['Dialer Name'].isin([None, '---'])]

## Check here the possibility of syncing automatically from the analytics

In [172]:
list = r"C:\Users\Akhil\Downloads\Total_D\Team_list"
os.chdir(list)
team = pd.read_csv('Team_list.csv')

In [173]:
for df in [team, combined]:
    df['Dialer Name'] = (
        df['Dialer Name']
        .astype(str)
        .str.replace(r'\s+', ' ', regex=True)
        .str.replace(r'@.*|\(.*', '', regex=True)
        .str.strip()  # strip handles both leading/trailing
        .str.lower()
    )
team = team.drop_duplicates(subset=['Dialer Name','Email'])
team.rename(columns={'Email': 'CRM ID'}, inplace=True)

In [ ]:
for df in [combined, team]:
    df['Dialer Name'] = df['Dialer Name'].astype(str)
combined = combined.merge(team, how='left', left_on='Dialer Name', right_on='Dialer Name')

In [ ]:
crm_id_null_df = combined[combined['CRM ID'].isnull()]
source = crm_id_null_df.groupby('Source')['Dialer Name'].unique()